# BERT - continue training from older notebook

In [1]:
!pip install pytorch_pretrained_bert rouge_score pyrouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 819.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 43.5 MB/s eta 0:00:0000:0100:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=fc60f06f6fa28133b6fc55892a8efd033167abf04c50ebd7b0b428e338b766e7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191604 sha256=de4a43810ed49acd998d18bad76aed81e53f0c1f73a78b4ccbdf7e1c91a62b8b
  Stored in directory: /root/.cache/pip/wheels/9a/67/12/c5dd8ef8b4152bb8789eafd2a74a734e2dc7bb9eae02b768e7
Successfully built rouge_score pyrouge
  Attempting 

In [2]:
inputs_dir = "/kaggle/input/bert-train-eval-2408-2023"
aiml_dir = inputs_dir + "/aiml-thesis"
bertsum_src_dir = aiml_dir + "/submods/bertsum/src"
raw_dials_dir = aiml_dir + "/data/raw_dialogues"
bertsum_dir = aiml_dir + "/submods/bertsum"
berstum_wk_dir = "/kaggle/working/aiml-thesis/submods/bertsum"

In [3]:
import os
import pandas as pd
import rouge_score
from rouge_score import rouge_scorer

In [4]:
%env BERTSUM_DIR={bertsum_dir}

env: BERTSUM_DIR=/kaggle/input/bert-train-eval-2408-2023/aiml-thesis/submods/bertsum


In [7]:
os.chdir("/kaggle/working")

In [8]:
!mkdir results models logs

In [9]:
!cp -r $BERTSUM_DIR/src .
!cp -r $BERTSUM_DIR/models/bert_transformer/* ./models

In [11]:
!ls -la

total 28
drwxr-xr-x  7 root root 4096 Aug 26 12:40 .
drwxr-xr-x  5 root root 4096 Aug 26 12:38 ..
drwxr-xr-x  2 root root 4096 Aug 26 12:38 .virtual_documents
drwxr-xr-x  2 root root 4096 Aug 26 12:40 logs
drwxr-xr-x  2 root root 4096 Aug 26 12:42 models
drwxr-xr-x  2 root root 4096 Aug 26 12:40 results
drwxr-xr-x 10 root root 4096 Aug 26 12:40 src


In [12]:
os.chdir("./src")

In [14]:
!python train.py -mode train -train_from "../models/model_step_35000.pt" -encoder transformer -dropout 0.1 -bert_data_path ./bert_pt/bertsumdata_ind -model_path /kaggle/working/models -result_path /kaggle/working/results -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 100 -batch_size 3000 -decay_method noam -train_steps 35100 -accum_count 2 -log_file /kaggle/working/logs/bert_transformer_2608_1354 -use_interval true -warmup_steps 10000 -ff_size 2048 -inter_layers 2 -heads 8

[2024-08-26 12:55:30,488 INFO] Device ID 0
[2024-08-26 12:55:30,488 INFO] Device cuda
[2024-08-26 12:55:30,652 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2024-08-26 12:55:30,653 INFO] extracting archive file ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpcwuj1jsg
[2024-08-26 12:55:34,319 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

/opt/conda/lib/python3.10/site-packages/pytorch_pretr

In [15]:
def calc_metrics_df(df, rouge_only=True):
    df_test_results_lst = []
    for idx, row in df.iterrows():
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
        rouge_scores = scorer.score(row['summary'], row['response'])
        rouges = dict()
        for k,v in rouge_scores.items():
            rouges[f"{k}_pr"] = round(v.precision, 4)
            rouges[f"{k}_re"] = round(v.recall, 4)
            rouges[f"{k}_f1"] = round(v.fmeasure, 4)
        if rouge_only == False:
            bert_scores = bertscore.compute(predictions=[row['response']], references=[row['summary']], lang="en")
            bert_scores.pop('hashcode')
            result = {
                **rouges,
                **{f"bertscore_{k[:2]}": round(v[0], 4) for k,v in bert_scores.items()},
                'meteor': round(meteor.compute(predictions=[row['response']], references=[row['summary']])['meteor'], 4),
            }
            row_res = {
                'conv_id': row['conv_id'],
                **result,
            }
           
        else:
            result = {
                **rouges,
            }
            row_res = {
                **result,
            }
        df_test_results_lst.append(row_res)
    return df_test_results_lst

In [17]:
!ls -la

total 64
drwxr-xr-x 10 root root  4096 Aug 26 12:40 .
drwxr-xr-x  8 root root  4096 Aug 26 12:54 ..
drwxr-xr-x  2 root root  4096 Aug 26 12:54 __pycache__
drwxr-xr-x  2 root root  4096 Aug 26 12:40 bert_pt
-rw-r--r--  1 root root  3895 Aug 26 12:40 distributed.py
drwxr-xr-x  5 root root  4096 Aug 26 12:40 json_dialogues
drwxr-xr-x  2 root root  4096 Aug 26 12:40 logs
drwxr-xr-x  3 root root  4096 Aug 26 12:40 models
drwxr-xr-x  3 root root  4096 Aug 26 12:40 others
drwxr-xr-x  3 root root  4096 Aug 26 12:40 prepro
-rw-r--r--  1 root root  2011 Aug 26 12:40 preprocess.py
drwxr-xr-x  5 root root  4096 Aug 26 12:40 tokenized_dialogues
-rw-r--r--  1 root root 12927 Aug 26 12:40 train.py


In [19]:
def evaluate_rouge(split_name):
    for root, _, files in os.walk("/kaggle/working", topdown=True):
        for results_file in files:
            if results_file.startswith("result") and results_file.endswith(".gold"):
                step_num = results_file[results_file.find("step")+4:-5]
                cand_filename = results_file[:-5] + ".candidate"
                with open(os.path.join(root, results_file), 'r') as gold_file:
                    with open(os.path.join(root, cand_filename), 'r') as cand_file:
                        df = pd.DataFrame(
                            {
                                'summary': gold_file.readlines(),
                                'response': cand_file.readlines(),
                            }
                        )
                results_df = pd.DataFrame(calc_metrics_df(df))
                results_df.to_csv(f"/kaggle/working/results/{split_name}_res_bertsum_s{step_num}_2408_1930.csv", index=False, header=True)
                print("CSV saved")
        break
    print("Finished saving evaluation results")

In [20]:
def test_model_checkpoints():
    for (root, _, files) in os.walk("/kaggle/working/models"):
        for filename in files:
            if filename.endswith(".pt"):
                step_num = filename[11:-3]
                %env TEST_FROM={os.path.join(root, filename)}
                !python train.py -mode test -test_from "$TEST_FROM" -bert_data_path "./bert_pt/bertsumdata_ind" -model_path "../models" -visible_gpus 0 -gpu_ranks 0 -batch_size 30000 -log_file "/kaggle/working/logs/bert_transformer_2608_1354_res" -result_path "/kaggle/working/results" -test_all true -report_rouge false -block_trigram true
#     for (root, _, files) in os.walk('/kaggle/working/models'):
#         for filename in files:
#             if filename.endswith(".pt"):
#                 step_num = filename[11:-3]
#                 %env TEST_FROM={os.path.join(root, filename)}
#                 !python train.py -mode test -test_from "$TEST_FROM" -bert_data_path "$BERTSUM_DIR/src/bert_pt/bertsumdata_ind" -model_path "/kaggle/working/models" -visible_gpus 0 -gpu_ranks 0 -batch_size 30000 -log_file "/kaggle/working/logs/bert_transformer_2508_1641_res" -result_path "/kaggle/working/results" -test_all true -report_rouge false -block_trigram true
    evaluate_rouge("test")

In [23]:
!cp $BERTSUM_DIR/bert_config_uncased_base.json ../

In [25]:
!ls -la ../

total 36
drwxr-xr-x  8 root root 4096 Aug 26 13:10 .
drwxr-xr-x  5 root root 4096 Aug 26 12:38 ..
drwxr-xr-x  2 root root 4096 Aug 26 12:38 .virtual_documents
-rw-r--r--  1 root root  313 Aug 26 13:10 bert_config_uncased_base.json
drwxr-xr-x  2 root root 4096 Aug 26 13:08 logs
drwxr-xr-x  2 root root 4096 Aug 26 12:57 models
drwxr-xr-x  2 root root 4096 Aug 26 12:40 results
drwxr-xr-x 10 root root 4096 Aug 26 12:40 src
drwxr-xr-x  2 root root 4096 Aug 26 12:54 temp


In [26]:
test_model_checkpoints()

env: TEST_FROM=/kaggle/working/models/model_step_35100.pt
[2024-08-26 13:10:43,720 INFO] Loading checkpoint from /kaggle/working/models/model_step_35100.pt
/kaggle/working/src/train.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on

In [ ]:
# Log to wandb
!pip install wandb

In [28]:
# Based on official docs: https://docs.wandb.ai/guides/track/log/working-with-csv
import wandb
os.environ["WANDB_PROJECT"] = "aiml-thesis-train"
run = wandb.init(settings=wandb.Settings(start_method="thread"))
for root, _, files in os.walk("/kaggle/working/results"):
    for res_file in files:
        file_path = os.path.join(root,res_file)
        df = pd.read_csv(file_path)
        tbl = wandb.Table(dataframe=df)
#         artif = wandb.Artifact(res_file[:-4], type="dataset")
#         artif.add(tbl, res_file[:-4] + "_table")
#         artif.add_file(file_path)
#         run.log_artifact(artif)
        run.log({"bert-results-table":tbl})

AttributeError: 'Run' object has no attribute 'log_table'